In [1]:
"""

主要是根据RAG的召回文档作为上下文，利用Agent回答query的问题，并给出结构化的输出：相关性评分、评分的依据或解释

"""

import os, json
from langchain_openai import ChatOpenAI



In [2]:


llm = ChatOpenAI(api_key="EMPTY", base_url="http://localhost:8000/v1", model="Qwen2.5-7B-Instruct-AWQ",)



In [13]:

from langchain_core.prompts import PromptTemplate


template = """给定question相关的上下文候选文档，请输出最相似的文档编号id、相关性评分（得分范围：1-10，得分越高越相似）以及相关性的依据。
context:
{context}

question: 
{question}

"""

prompt = PromptTemplate.from_template(template)

# 候选相关文档，可以是RAG召回的文档、embedding召回的文档、BM25召回的文档。

question = "中国南北的菜系文化有什么不同"

candidated_doc_list = ["这篇文章介绍了中国八大菜系的起源、特色和代表性菜肴，包括川菜、粤菜、鲁菜等，通过对比不同菜系的特点，间接反映了南北饮食文化的差异。",
                       "本文探讨了中国传统节日中的饮食习俗，如春节的饺子、端午节的粽子等，这些习俗在南北地区有着不同的表现形式，从而体现了南北饮食文化的差异。",
                       "这篇文章研究了中国南北地区主食（如北方的面食和南方的米饭）的差异，并分析了这些差异对人们健康的影响，从营养学的角度探讨了饮食文化的不同。",
                       "本文比较了中国南北地区的酒文化，包括酒的种类、饮酒习惯和酒在社交中的作用，这些差异反映了南北饮食文化的不同侧面"
                      ]
context = "\n".join([str(index+1) + ":" + doc for index, doc in enumerate(candidated_doc_list)])
print(context)

1:这篇文章介绍了中国八大菜系的起源、特色和代表性菜肴，包括川菜、粤菜、鲁菜等，通过对比不同菜系的特点，间接反映了南北饮食文化的差异。
2:本文探讨了中国传统节日中的饮食习俗，如春节的饺子、端午节的粽子等，这些习俗在南北地区有着不同的表现形式，从而体现了南北饮食文化的差异。
3:这篇文章研究了中国南北地区主食（如北方的面食和南方的米饭）的差异，并分析了这些差异对人们健康的影响，从营养学的角度探讨了饮食文化的不同。
4:本文比较了中国南北地区的酒文化，包括酒的种类、饮酒习惯和酒在社交中的作用，这些差异反映了南北饮食文化的不同侧面


In [14]:

from typing import Optional, List, Annotated

from pydantic import BaseModel, Field


# https://api.python.langchain.com/en/latest/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI.extra_body
# 上述中主要看到的是extra_body, 一个作用是用来定义输出的格式
# 可以参考： https://python.langchain.com/docs/how_to/structured_output/#the-with_structured_output-method

class SelectAnswer(BaseModel):
    answer_id: str = Field(description="问题与候选文档最相似的文档编号ID")
    answer_score: float = Field(description="问题与最相关文档的得分，得分范围1-10，分值越大，相关性越高")
    answer_explain: str = Field(description="解释问题与该文档最相似的原因")

llm_with_structured_output = llm.with_structured_output(SelectAnswer)

qa_chain = prompt | llm

qa_chain_with_structure_output = prompt | llm_with_structured_output



In [16]:

print("----------------------通过指令prompt来控制输出------------------")

inputs = {"question": question, "context": context}

response_qa = qa_chain.invoke(inputs)
print(response_qa.content.split("\n"))



----------------------通过指令prompt来控制输出------------------
['文档编号：1', '相关性评分：9', '依据：这篇文章直接讨论了中国八大菜系的起源、特色和代表性菜肴的区别，特别是强调了川菜、粤菜、鲁菜等南北菜系的特点差异，间接反映了南北饮食文化的差异，与问题中关于中国南北菜系文化不同的询问高度相关。']


In [17]:
print("----------------------通过指定pydantic或json schema来控制输出------------------")

response_qa_with_structured_output = qa_chain_with_structure_output.invoke(inputs)
print(response_qa_with_structured_output)


----------------------通过指定pydantic或json schema来控制输出------------------
answer_id='1' answer_score=8.0 answer_explain='这篇文章详细介绍了中国八大菜系的起源、特色和代表性菜肴，明确区分了川菜、粤菜、鲁菜等南北菜系的特点，深入探讨了不同菜系背后蕴含的南北饮食文化的差异，符合问题关于中国南北菜系文化不同的需求。'


In [18]:
print(response_qa_with_structured_output.answer_id)
print(response_qa_with_structured_output.answer_score)
print(response_qa_with_structured_output.answer_explain)

1
8.0
这篇文章详细介绍了中国八大菜系的起源、特色和代表性菜肴，明确区分了川菜、粤菜、鲁菜等南北菜系的特点，深入探讨了不同菜系背后蕴含的南北饮食文化的差异，符合问题关于中国南北菜系文化不同的需求。
